In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
import numpy as np
from matplotlib.patches import Polygon, Circle
from matplotlib.offsetbox import OffsetImage
from matplotlib.image import imread
import os

# Charger les données Excel
excel_file = "triangle_data.xlsx"  # Remplacez par le nom de votre fichier Excel
data = pd.read_excel(excel_file)

# Lire les angles des colonnes B et C (ligne 1)
A = pd.to_numeric(data.iloc[0, 1], errors='coerce')  # Angle A en radians (colonne B)
B = pd.to_numeric(data.iloc[0, 2], errors='coerce')  # Angle B en radians (colonne C)

# Calculer le troisième angle C
C = np.pi - A - B

# Vérification des angles
if np.isnan(A) or np.isnan(B) or C <= 0:
    raise ValueError("Les angles doivent être valides et respecter la condition A + B + C = π.")

angles = np.array([A, B, C])

# Lire les coordonnées barycentriques et les données associées
points = data.iloc[0:, 1:4].values.astype(float)  # Coordonnées barycentriques
colors = data.iloc[0:, 4:7].values / 255  # Normaliser les couleurs en RGB
comments = data.iloc[0:, 7].astype(str).values  # Assurer que les commentaires sont des chaînes

# Filtrer les données invalides
valid_points = []
valid_colors = []
valid_comments = []
for i, bary_coords in enumerate(points):
    if np.isclose(np.sum(bary_coords), 1.0) and np.all(bary_coords >= 0):
        valid_points.append(bary_coords)
        valid_colors.append(colors[i])
        valid_comments.append(comments[i])
    else:
        print(f"Ligne {i + 2} ignorée : coordonnées barycentriques invalides {bary_coords}")

# Convertir en tableaux numpy
valid_points = np.array(valid_points)
valid_colors = np.array(valid_colors)
valid_comments = np.array(valid_comments)

# Calculer les sommets du triangle en coordonnées cartésiennes
triangle = np.array([
    [0, 0],
    [1, 0],
    [np.cos(A), np.sin(A)]
])

# Ajouter des labels pour A, B et C
triangle_labels = ['Constrainsts', 'Context', 'Contiuous Improvements']

# Convertir les coordonnées barycentriques en cartésiennes
def barycentric_to_cartesian(triangle, bary_coords):
    return np.dot(bary_coords, triangle)

cartesian_coords = np.array([barycentric_to_cartesian(triangle, bc) for bc in valid_points])

# Calculer le centre du cercle inscrit (incenter) et son rayon
def calculate_incircle(triangle):
    # Longueurs des côtés
    a = np.linalg.norm(triangle[1] - triangle[2])  # Opposé à A
    b = np.linalg.norm(triangle[0] - triangle[2])  # Opposé à B
    c = np.linalg.norm(triangle[0] - triangle[1])  # Opposé à C

    # Coordonnées du centre du cercle inscrit
    incenter = (a * triangle[0] + b * triangle[1] + c * triangle[2]) / (a + b + c)

    # Demi-périmètre
    semi_perimeter = (a + b + c) / 2

    # Aire du triangle
    area = 0.5 * np.abs(
        triangle[0, 0] * (triangle[1, 1] - triangle[2, 1]) +
        triangle[1, 0] * (triangle[2, 1] - triangle[0, 1]) +
        triangle[2, 0] * (triangle[0, 1] - triangle[1, 1])
    )

    # Rayon
    radius = area / semi_perimeter

    return incenter, radius

incenter, radius = calculate_incircle(triangle)

# Charger l'image miniature de l'avion
image_path = "avion_s.png"
if not os.path.exists(image_path):
    raise FileNotFoundError(f"Le fichier {image_path} n'existe pas dans le répertoire actuel.")
plane_image = plt.imread(image_path)

# Créer l'animation
fig, ax = plt.subplots(figsize=(4, 4))  # Taille personnalisable
ax.axis('off')  # Supprimer les axes
ax.set_aspect('equal', adjustable='datalim')  # Assurer une échelle égale

# Ajouter le triangle
triangle_patch = Polygon(triangle, edgecolor='#D3D3D3', fill=None)
ax.add_patch(triangle_patch)

# Ajouter des labels pour A, B et C
for i, (x, y) in enumerate(triangle):
    ax.text(x, y, triangle_labels[i], fontsize=12, ha="center", va="center", color="blue")

# Dessiner le cercle inscrit
circle = Circle(incenter, radius, edgecolor="gray", fill=False, linestyle="--", lw=1)
ax.add_patch(circle)

# Initialiser les éléments de l'animation
point, = ax.plot([], [], 'o', markersize=5)
comment_text = ax.text(0, 0, "", fontsize=14, color="black", ha="center")
plane = ax.imshow(plane_image, extent=[0, 0, 0, 0], visible=False)  # Avion caché au départ
progress_line, = ax.plot([], [], lw=2, color="black")  # Barre de progression
arrow_lines = []  # Stocker les flèches en pointillé pour chaque point

def update(frame):
    total_points = len(cartesian_coords)
    point_index = frame // 70 # 70 frames par point (animation + pause)
    point_frame = frame % 100

    if point_index >= total_points:
        return point, comment_text, progress_line, *arrow_lines  # Arrêter après le dernier point

    x_start, y_start = cartesian_coords[point_index]  # Départ du point
    x_end, y_end = incenter  # Arrivée au centre du cercle inscrit

    # Interpolation du mouvement
    t = point_frame / 100
    x = x_start + t * (x_end - x_start)
    y = y_start + t * (y_end - y_start)

    # Mettre à jour le point
    point.set_data([x], [y])
    point.set_color(valid_colors[point_index])

    # Mettre à jour le texte (commentaire)
    comment_text.set_position((x, y + 0.05))  # Légèrement au-dessus du point
    comment_text.set_text(valid_comments[point_index])
    comment_text.set_color(valid_colors[point_index])

    # Mettre à jour la barre de progression
    progress_line.set_data([x_start, x], [y_start, y])
    progress_line.set_color(valid_colors[point_index])

    # Ajouter une flèche en pointillé fine du point au centre si non déjà ajoutée
    if len(arrow_lines) <= point_index:
        arrow = ax.plot(
            [x_start, x_end],
            [y_start, y_end],
            linestyle="dotted",
            lw=0.5,
            color=valid_colors[point_index]
        )[0]
        arrow_lines.append(arrow)

    # Afficher l'avion une fois au centre
    if t >= 1.0:  # Arrivé au centre
        plane.set_extent([x_end - 0.01, x_end + 0.01, y_end - 0.01, y_end + 0.01])
        plane.set_visible(True)
    else:
        plane.set_visible(False)

    return point, comment_text, progress_line, *arrow_lines

# Créer l'animation
frames_per_point = 70  # 70 frames pour le déplacement
total_frames = frames_per_point * len(cartesian_coords)
animation = FuncAnimation(
    fig, update, frames=total_frames, interval=35, blit=False, repeat=False
)

# Enregistrer l'animation au format GIF
output_gif = "triangle_points_with_progress_and_arrows_used_.gif"
animation.save(output_gif, writer='pillow', dpi=100)

print(f"Animation enregistrée sous {output_gif}")
